In [38]:
import sys
print(sys.executable)

/home/fatnaoui/tools/miniforge3/envs/fasttext/bin/python


In [39]:
from datasets import load_dataset
import nltk
from nltk.tokenize import sent_tokenize
from pathlib import Path
from tqdm import tqdm

from helpers.filtering import looks_like_clean_english

In [40]:
nltk.download("punkt_tab")

out_file = Path("en_data.txt")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/fatnaoui/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [36]:
print("Loading English Wikipedia dataset...")
dataset = load_dataset("wikimedia/wikipedia", "20231101.en", split="train", streaming=True)

Loading English Wikipedia dataset...


In [ ]:
min_words = 3 # min words in each example
n_sentences = 0
n_articles = 0
n_words = 0
TARGET_ARTICLES = 10 # 20_000
TARGET_SENTENCES = 50_000
TARGET_WORDS = 10_000

with out_file.open("w", encoding="utf-8") as f_out:
    for example in tqdm(dataset, desc="Processing articles"):

        article_not_seen = True

        # if n_articles >= TARGET_ARTICLES or n_sentences >= TARGET_SENTENCES or n_words >= TARGET_WORDS:
        #     break

        text = example["text"]
        if not text:
            continue

        sentences = sent_tokenize(text)

        for sent in sentences:
            if n_sentences >= TARGET_SENTENCES:
                break

            sent = sent.strip()
            if not sent:
                continue

            word_count = len(sent.split())
            if word_count < min_words:
                continue

            # our extra "robust English" filter
            if not looks_like_clean_english(sent):
                continue

            # normalize
            sent = sent.lower()

            f_out.write(sent.replace("\n", " ") + "\n")
            n_words += word_count
            n_sentences += 1
            if article_not_seen:
                n_articles += 1
                article_not_seen = False

print(f"Done. Saved {n_articles} articles, {n_words} words, and {n_sentences} sentences to: {out_file}")


Processing articles: 0it [00:00, ?it/s]

Processing articles: 2it [01:23, 41.91s/it]

Done. Saved 2 articles, 10590 words, and 455 sentences to: en_data.txt
